***FCIM.FIA - Fundamentals of Artificial Intelligence***

> **Lab 3:** *Domains, Constraints* \\
> **Performed by:** *Bajenov Sevastian*, group *FAF-213* \\
> **Verified by:** Elena Graur, asist. univ.

## Imports and Utils

Create a virtual environment, install all the necessary dependencies so that you can run the notebook using your virtual environment as a kernel.

In [1]:
# pip install -r requirements.txt

## Conclusions:

## Acknowledgements


## Bibliography: